In [7]:
from ultralytics import YOLO
from ultralytics.utils.metrics import mask_iou
from torch import bitwise_or, zeros, Tensor
import cv2
import numpy as np
import pandas as pd

model = YOLO('runs/segment/train2/weights/best.pt')

In [8]:
def read_and_normalize_mask(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    image = cv2.normalize(image, None, 0, 1, cv2.NORM_MINMAX)

    _, mask = cv2.threshold(image, 0.5, 1, cv2.THRESH_BINARY)

    mask = mask.reshape(-1, int(image.shape[1] * image.shape[0]))

    return Tensor(mask)

def get_iou(yolo_model, img_path):
    results = yolo_model(img_path)
    true_mask = read_and_normalize_mask(img_path.replace("images", "masks"))
    
    assert len(results) == 1, "support only len == 1"
    for r in results:
        if isinstance(r.masks, type(None)): return None
        masks = r.masks.data.reshape(-1, 256*256).cpu()
        masks = np.any(np.array(masks), axis=0)
        masks = masks.reshape(-1, 256*256)
        print(masks)
        masks = Tensor(masks)
    
        return mask_iou(true_mask, masks)


In [9]:
%%capture

# get an average iou
import os
import numpy as np

image_directory = "./data/ar-test/images/"

iou_values = []
for image_path in os.listdir(image_directory):
    if image_path.endswith(".png"):
        result = get_iou(model, os.path.join(image_directory, image_path))
        if result != None:
            iou = result.tolist()[0][0]
            iou_values.append(iou)

average_iou = np.mean(iou_values)
median = np.median(iou_values)


image 1/1 /home/artipark/Studenti/BSH/bakalarka-yolov8/data/ar-test/images/2016_01_31__22_59_58_34__SDO_AIA_AIA_171.png: 256x256 3 active regions, 11.9ms
Speed: 0.3ms preprocess, 11.9ms inference, 0.7ms postprocess per image at shape (1, 3, 256, 256)

image 1/1 /home/artipark/Studenti/BSH/bakalarka-yolov8/data/ar-test/images/2016_09_22__22_59_46_35__SDO_AIA_AIA_171.png: 256x256 4 active regions, 12.0ms
Speed: 0.2ms preprocess, 12.0ms inference, 0.6ms postprocess per image at shape (1, 3, 256, 256)

image 1/1 /home/artipark/Studenti/BSH/bakalarka-yolov8/data/ar-test/images/2016_06_13__23_00_10_34__SDO_AIA_AIA_171.png: 256x256 3 active regions, 12.0ms
Speed: 0.2ms preprocess, 12.0ms inference, 0.6ms postprocess per image at shape (1, 3, 256, 256)

image 1/1 /home/artipark/Studenti/BSH/bakalarka-yolov8/data/ar-test/images/2016_10_09__23_00_10_35__SDO_AIA_AIA_171.png: 256x256 5 active regions, 11.9ms
Speed: 0.2ms preprocess, 11.9ms inference, 0.6ms postprocess per image at shape (1, 3, 25

In [10]:
%%capture cell
val_res = model.val(data='./ar.yaml', imgsz=256, split="test")

Ultralytics YOLOv8.0.145 🚀 Python-3.7.16 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce RTX 4090, 24195MiB)
val: New cache created: /home/artipark/Studenti/BSH/bakalarka-yolov8/data/ar-test/images-col.cache
                   all       1072       1798      0.367      0.338      0.281     0.0773      0.269       0.32      0.212     0.0477
Speed: 0.1ms preprocess, 2.0ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/segment/val2


In [11]:
dice = val_res.seg.f1
print("Dice score:", dice[0])
print("IoU:", average_iou)

#  results for ch colorful images
# Dice score: 0.5807742979901169
# IoU: 0.7006662135058757

# results for ch grayscale
# Dice score: 0.5563138232924477
# IoU: 0.6986303846845785

# results for ar colorful
# Dice score: 0.2744738541877972
# IoU: 0.3191702537726108

# results for ar grayscale
# Dice score: 0.2924788789771119
# IoU: 0.32751943564349206

Dice score: 0.2924788789771119
IoU: 0.32751943564349206
